In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
CACHE = {}

In [ ]:
WIKI_ROOT = 'https://fr.wikipedia.org'

In [ ]:
ROWS_THRESHOLD = 7

In [ ]:
COLUMN_NAMES = set(['ville', 'transcription'])

In [ ]:
def my_get(url):
    if url in CACHE:
        return CACHE[url]
    soup = BeautifulSoup(requests.get(WIKI_ROOT + url).text, "lxml")
    CACHE[url] = soup
    return soup

In [ ]:
s = my_get('/wiki/Listes_des_villes_du_monde')

In [ ]:
def get_name_column_index(row):
    header_cols = row.findAll('th')
    if len(header_cols) < 1:
        header_cols = row.findAll('td')
    for idx, header in enumerate(header_cols):
        if header.text.lower() in COLUMN_NAMES:
            print('index of column with city name is', idx)
            return idx

In [ ]:
def scrape_cities(country_soup):
    tables = country_soup.findAll('table')
    cities = []
    for table in tables:
        rows = table.findAll('tr')
        if rows is None:
            continue
        if len(rows) > ROWS_THRESHOLD:
            idx = get_name_column_index(rows[0])
            for row in rows[1:]:
                city = row.findAll('td')[idx].text
                cities.append(city)
            break
        else:
            print('No table with more that {} rows was found in {}'.format(ROWS_THRESHOLD, country_soup.find('h1').text))
    return cities

In [ ]:
def scrape_countries(lis):
    countries = []
    for li in lis:
        country = {}
        anchor = li.find('a')
        country['name'] = anchor.text
        country['url'] = anchor['href']
        if 'redlink' in country['url']:
            print(country['name'], 'does not exist')
            continue
        print('Scraping', country['name'])
        country['cities'] = scrape_cities(BeautifulSoup(my_get(country['url']).text, "lxml"))
        countries.append(country)
    return countries

In [ ]:
WORLD = {}

In [ ]:
for header2 in s.findAll("h2"):# class_="mw-headline"):
    continent_tag = header2.find('a')
    if continent_tag is not None:
        continent_name = continent_tag.text
        country_list = header2.find_next_sibling().findAll('li')
        WORLD[continent_name] = scrape_countries(country_list)

In [ ]:
WORLD

In [ ]:
import json

In [ ]:
with open('world_scraping.json', 'w') as f:
    json.dump(WORLD, f, indent=2)